In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

#Import Libraries
import tensorflow as tf
import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
import datetime
import time

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

from sklearn.metrics import recall_score, precision_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from models import save_model, generate_model_CNN, plot_performance
from generators import generator, decode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
save_dir = 'plots'
if not os.path.exists(save_dir):
        os.makedirs(save_dir)

# X_load = np.load('../data/processed_data/m4019_session1/X.npy')
# Y_load_flat = np.load('../data/processed_data/m4019_session1/y.npy')


# enc = OneHotEncoder(categories='auto')
# Y_load = enc.fit_transform(Y_load_flat.reshape(-1, 1) ,).toarray()

# split_index = int(X_load.shape[0]*0.8)
    
# X_train = X_load[:split_index]
# Y_train = Y_load[:split_index]
# # V_train = V_load[:split_index]

# X_val = X_load[split_index:]
# Y_val = Y_load[split_index:]
# # V_val = V_load[split_index:]


X_train = np.load('../data/processed_data/X_train.npy')
Y_train_flat = np.load('../data/processed_data/y_train.npy')

enc = OneHotEncoder(categories='auto')
Y_train = enc.fit_transform(Y_train_flat.reshape(-1, 1) ,).toarray()


X_test = np.load('../data/processed_data/X_test.npy')
Y_test_flat = np.load('../data/processed_data/y_test.npy')

enc = OneHotEncoder(categories='auto')
Y_test = enc.fit_transform(Y_test_flat.reshape(-1, 1) ,).toarray()

In [6]:
batch_size = 500
time_window = 100
n_dim = 1
class_weight = {0: 1., 1: 50.,}
predict_early = 0

input_shape = (time_window, n_dim)

In [7]:
train_generator = generator(X_train, Y_train, batch_size, time_window, predict_early=predict_early)
validate_generator = generator(X_val, Y_val, batch_size, time_window, predict_early=predict_early)

NameError: name 'X_val' is not defined

In [ ]:
model = generate_model_CNN(input_shape)

print(model.summary())

In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=1000,
                              epochs=15, 
                              verbose=1,
                              validation_data=validate_generator,
                              validation_steps=1000,
                              class_weight=class_weight, )

save_model(model, name=predict_early)

In [ ]:
plot_performance(history)

In [ ]:
validate_generator = generator(X_val, Y_val, batch_size*100, time_window)

In [ ]:
X_trial, y_trial, = next(validate_generator)
res = model.predict(X_trial)

In [ ]:
threshold=0.9999
y_true = decode(y_trial, threshold=threshold)
y_pred = decode(res, threshold=threshold)

weights = np.zeros_like(y_true)
weights[y_true==0] = 1
weights[y_true==1] = 100


In [ ]:
from sklearn.metrics import balanced_accuracy_score, recall_score, classification_report, confusion_matrix

print(balanced_accuracy_score(y_true, y_pred, adjusted=True))
print(recall_score(y_true, y_pred, average='weighted'))

target_names = ['no ripple', 'ripple',]
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
conv_matrix = confusion_matrix(y_true, y_pred)
print(conv_matrix)
conv_matrix = conv_matrix.astype('float') / conv_matrix.sum(axis=1)[:, np.newaxis]
print(conv_matrix)

In [ ]:
plt.figure()
plt.plot(y_true)
plt.plot(y_pred+1)
plt.show()

In [ ]:
plt.figure()
plt.plot(X_load)
plt.plot(Y_load_flat)
plt.show()

In [ ]:
Y_load.shape

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit
# tscv = TimeSeriesSplit(n_splits=5)
# for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)